# Vortex-Sentiment Adaptive Volatility (VSAV) Strategy

### Data Oracles - Competition B



## Importing Necessary Libraries for Analysis

In [788]:
import yfinance as yf  # For downloading financial data
import numpy as np      # For numerical operations
import pandas as pd     # For data manipulation
import requests # For downloading the API data
import numpy as np 
import plotly.graph_objects as go
import plotly.express as px # Import the Plotly Express module for interactive visualization
import json
import vectorbt as vbt

## Data Collection

### Fetch daily OHLCV data 

In [753]:
# Data for the TSLA, XLY, and SPY tickers is retrieved from the Yahoo Finance library, covering the period from January 1, 2019, 
# to March 5, 2025.
tsla = yf.download('TSLA', start='2019-01-01', end='2025-03-05') 
xly = yf.download('XLY', start='2019-01-01', end='2025-03-05')
spy = yf.download('SPY', start='2019-01-01', end='2025-03-05')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [754]:
# Displays a summary of the TSLA DataFrame, including column names, data types, non-null counts, and memory usage.
tsla.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1551 entries, 2019-01-02 to 2025-03-04
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   (Close, TSLA)   1551 non-null   float64
 1   (High, TSLA)    1551 non-null   float64
 2   (Low, TSLA)     1551 non-null   float64
 3   (Open, TSLA)    1551 non-null   float64
 4   (Volume, TSLA)  1551 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 72.7 KB


In [755]:
# Displays a summary of the XLY DataFrame, including column names, data types, non-null counts, and memory usage.
xly.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1551 entries, 2019-01-02 to 2025-03-04
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   (Close, XLY)   1551 non-null   float64
 1   (High, XLY)    1551 non-null   float64
 2   (Low, XLY)     1551 non-null   float64
 3   (Open, XLY)    1551 non-null   float64
 4   (Volume, XLY)  1551 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 72.7 KB


In [756]:
# Displays a summary of the SPY DataFrame, including column names, data types, non-null counts, and memory usage.
spy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1551 entries, 2019-01-02 to 2025-03-04
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   (Close, SPY)   1551 non-null   float64
 1   (High, SPY)    1551 non-null   float64
 2   (Low, SPY)     1551 non-null   float64
 3   (Open, SPY)    1551 non-null   float64
 4   (Volume, SPY)  1551 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 72.7 KB


### Fetch sentiment scores from the API

In [757]:
# Defines the API endpoint URL for retrieving news sentiment data related to Tesla (TSLA) 
# from the Alpha Vantage service. The query specifies the function type, date range, result limit, 
# targeted ticker symbol, and a valid API key.
###url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&date_from=20250101T0130&date_to=20250301T0130&limit=1000&tickers=TSLA&apikey=PNM5EHRALIOT1CKJ'

# Sends a GET request to the specified URL to initiate the API call.
###response = requests.get(url)

# Evaluates whether the API call was successful based on the HTTP response status code.
###if response.status_code == 200:
    # Parses the JSON response and extracts the 'feed' section containing sentiment data.
    ###sentiment_data = response.json()
    
    # Converts the extracted sentiment feed into a DataFrame for further analysis or visualization.
    ###sentiment_df = pd.DataFrame(sentiment_data['feed']) 
    
    # Displays the first five rows of the sentiment DataFrame to provide an overview of the retrieved content.
    ###print(sentiment_df.head())
####else:
    # Prints an error message if the API request was unsuccessful.
    ###print("API call failed:", response.status_code)

# Independently parses the full JSON response and prints its contents for inspection or debugging purposes.
###sentiment_json = response.json()
###print(sentiment_json)

## Indicator Calculation

### Compute VI+ and VI-

In [758]:
# Defines a function to calculate the Vortex Indicator (VI) for a given DataFrame and ticker symbol.
# The calculation uses a default lookback period of 14 days unless specified otherwise.
def calculate_vortex(df, value, n=14):
    # Extracts the high, low, and close price series for the specified ticker.
    high = df[("High", value)]
    low = df[("Low", value)]
    close = df[("Close", value)]

    # Calculates the Vortex Movement values:
    # VM+ = absolute difference between today's high and yesterday's low
    # VM− = absolute difference between today's low and yesterday's high
    vm_plus = abs(high - low.shift(1))     # |Today's High – Yesterday’s Low|
    vm_minus = abs(low - high.shift(1))    # |Today's Low – Yesterday’s High|

    # Computes the True Range (TR) as the maximum of:
    # - High - Low
    # - Absolute difference between High and Previous Close
    # - Absolute difference between Low and Previous Close
    tr = pd.concat([
        high - low,
        abs(high - close.shift(1)),
        abs(low - close.shift(1))
    ], axis=1).max(axis=1)

    # Applies a rolling window to compute the n-period sum of VM+ and VM− values
    # and the corresponding True Range values.
    sum_vm_plus = vm_plus.rolling(window=n).sum()
    sum_vm_minus = vm_minus.rolling(window=n).sum()
    sum_tr = tr.rolling(window=n).sum()

    # Calculates the Vortex Indicator components:
    # VI+ = sum of VM+ over n periods divided by sum of TR over n periods
    # VI− = sum of VM− over n periods divided by sum of TR over n periods
    vi_plus = sum_vm_plus / sum_tr
    vi_minus = sum_vm_minus / sum_tr

    # Returns the VI+ and VI− series as output.
    return vi_plus, vi_minus

In [759]:
# Calculates the Vortex Indicator values for TSLA and stores the results as new columns in the DataFrame.
tsla['VI+'], tsla['VI-'] = calculate_vortex(tsla, 'TSLA')

# Calculates the Vortex Indicator values for XLY and stores the results as new columns in the DataFrame.
xly['VI+'], xly['VI-'] = calculate_vortex(xly, 'XLY')

# Calculates the Vortex Indicator values for SPY and stores the results as new columns in the DataFrame.
spy['VI+'], spy['VI-'] = calculate_vortex(spy, 'SPY')

In [760]:
# Displays the first 20 rows of the TSLA DataFrame to provide an initial overview of its structure and content with the new function applied.
tsla.head(20)

Price,Close,High,Low,Open,Volume,VI+,VI-
Ticker,TSLA,TSLA,TSLA,TSLA,TSLA,,
Date,,,,,,,
2019-01-02,20.674667,21.008667,19.920000,20.406668,174879000,NaN,NaN
2019-01-03,20.024000,20.626667,19.825333,20.466667,104478000,NaN,NaN
2019-01-04,21.179333,21.200001,20.181999,20.400000,110911500,NaN,NaN
2019-01-07,22.330667,22.449333,21.183332,21.448000,113268000,NaN,NaN
2019-01-08,22.356667,22.934000,21.801332,22.797333,105127500,NaN,NaN
2019-01-09,22.568666,22.900000,22.098000,22.366667,81493500,NaN,NaN
2019-01-10,22.997999,23.025999,22.119333,22.293333,90846000,NaN,NaN
2019-01-11,23.150667,23.227333,22.584667,22.806000,75586500,NaN,NaN


### Calculate Volume-Weighted Sentiment 

In [761]:
# Load the sentiment JSON file from local storage
with open("TSLA_sentiment.json", "r") as f:
    sentiment_json = json.load(f)

# Extract the "feed" list from the top-level JSON dictionary.
# This section contains the array of sentiment articles or entries.
sentiment_feed = sentiment_json.get("feed", [])

# Initialize an empty list to hold cleaned and structured sentiment data
sentiment_data = []

# Iterate through each item in the sentiment feed to extract relevant fields
for item in sentiment_feed:
    try:
        sentiment_data.append({
            # Convert the timestamp to pandas datetime for proper indexing
            "time_published": pd.to_datetime(item["time_published"]),
            # Convert the sentiment score string to float
            "sentiment_score": float(item["overall_sentiment_score"]),
            # Store the sentiment label (e.g., Positive, Neutral, Negative)
            "sentiment_label": item["overall_sentiment_label"],
        })
    except (KeyError, ValueError, TypeError):
        # Skip malformed or incomplete entries that raise an error
        continue

# Convert the structured list of dictionaries into a pandas DataFrame
sentiment_df = pd.DataFrame(sentiment_data)

# Set the 'time_published' column as the DataFrame index to enable time-series operations
sentiment_df.set_index("time_published", inplace=True)

# Display the first few rows of the DataFrame to verify content and structure
print(sentiment_df.head())

# Output a summary of the DataFrame structure, including column types and memory usage
print(sentiment_df.info())

                     sentiment_score   sentiment_label
time_published                                        
2025-03-01 00:00:18         0.225994  Somewhat-Bullish
2025-02-28 20:33:00        -0.098739           Neutral
2025-02-28 20:07:43        -0.041235           Neutral
2025-02-28 20:07:36        -0.038786           Neutral
2025-02-28 18:24:25         0.021961           Neutral
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 681 entries, 2025-03-01 00:00:18 to 2025-01-31 13:58:04
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sentiment_score  681 non-null    float64
 1   sentiment_label  681 non-null    object 
dtypes: float64(1), object(1)
memory usage: 16.0+ KB
None


In [762]:
# Initialize an empty list to store processed sentiment records
sentiment_data = []

# Iterate through each news item in the 'feed' section of the JSON object
for news_item in sentiment_json.get("feed", []):
    # Append a dictionary with selected and transformed fields to the sentiment list
    sentiment_data.append({
        # Convert the time of publication to datetime format
        "time_published": pd.to_datetime(news_item["time_published"]),
        # Extract the sentiment score (as-is; conversion to float may be handled separately if needed)
        "sentiment_score": news_item["overall_sentiment_score"],
        # Extract the sentiment label (e.g., Positive, Neutral, Negative)
        "sentiment_label": news_item["overall_sentiment_label"],
    })

# Convert the list of dictionaries into a pandas DataFrame
sentiment_data = pd.DataFrame(sentiment_data)

In [763]:
# Sort the DataFrame by publication time in ascending order for chronological analysis
sentiment_data['time_published'].sort_values(ascending=True)

680   2025-01-31 13:58:04
679   2025-01-31 14:05:00
678   2025-01-31 14:05:31
677   2025-01-31 14:31:00
676   2025-01-31 15:13:26
              ...        
4     2025-02-28 18:24:25
3     2025-02-28 20:07:36
2     2025-02-28 20:07:43
1     2025-02-28 20:33:00
0     2025-03-01 00:00:18
Name: time_published, Length: 681, dtype: datetime64[ns]

In [764]:
# Convert the 'time_published' column to only retain the date portion (drop time-of-day)
sentiment_data['time_published'] = sentiment_data['time_published'].dt.date

In [765]:
# Filter sentiment data to retain only those records that match dates present in the TSLA index
sentiment_scores_filtered = sentiment_data[
    pd.to_datetime(sentiment_data['time_published']).isin(tsla.index)
]

# Group the filtered data by publication date and calculate the average sentiment score per day
sentiment_scores_filtered = sentiment_scores_filtered.groupby('time_published')['sentiment_score'].mean().reset_index()

In [766]:
# Fix the multi-level column issue by selecting the 'Volume' column and resetting its name
tsla_volume = tsla[('Volume', 'TSLA')].rename('Volume')

# Ensure the index of tsla_volume is a column and convert it to match the type of time_published
tsla_volume = tsla_volume.reset_index()
tsla_volume['Date'] = pd.to_datetime(tsla_volume['Date'])

In [767]:
# Convert 'time_published' in the sentiment data to datetime to match volume data type
sentiment_scores_filtered['time_published'] = pd.to_datetime(sentiment_scores_filtered['time_published'])

# Perform an inner merge between sentiment scores and volume data based on matching dates
merged_data = pd.merge(
    tsla_volume,
    sentiment_scores_filtered,
    left_on='Date',
    right_on='time_published',
    how='inner'
)

In [768]:
# Compute the weighted sentiment by multiplying raw sentiment by trading volume
merged_data['Weighted_Sentiment'] = merged_data['Volume'] * merged_data['sentiment_score']

# Calculate a 5-day rolling average of the weighted sentiment to smooth short-term noise
merged_data['5_day_avg_sentiment'] = merged_data['Weighted_Sentiment'].rolling(window=5).mean()

# Define a binary condition for when the average sentiment is positive
merged_data['Buy_Condition'] = merged_data['5_day_avg_sentiment'] > 0

# Normalize the rolling sentiment score by average volume to allow comparability across scales
merged_data['5_day_avg_sentiment_norm'] = (
    merged_data['5_day_avg_sentiment'] / merged_data['Volume'].mean()
)

In [769]:
merged_data

,Date,Volume,time_published,sentiment_score,Weighted_Sentiment,5_day_avg_sentiment,Buy_Condition,5_day_avg_sentiment_norm
0,2025-01-31,83568200,2025-01-31,0.194614,1.626354e+07,NaN,False,NaN
1,2025-02-03,93732100,2025-02-03,0.129243,1.211426e+07,NaN,False,NaN
2,2025-02-04,57072200,2025-02-04,0.173107,9.879602e+06,NaN,False,NaN
3,2025-02-05,57223300,2025-02-05,0.136874,7.832396e+06,NaN,False,NaN
4,2025-02-06,77918200,2025-02-06,0.118095,9.201782e+06,1.105832e+07,True,0.132787
5,2025-02-07,70298300,2025-02-07,0.133871,9.410915e+06,9.687792e+06,True,0.116330
6,2025-02-10,77514900,2025-02-10,0.152754,1.184073e+07,9.633086e+06,True,0.115673
7,2025-02-11,118543400,2025-02-11,0.164455,1.949505e+07,1.155618e+07,True,0.138766
8,2025-02-12,105382700,2025-02-12,0.147806,1.557620e+07,1.310494e+07,True,0.157363
9,2025-02-13,89441500,2025-02-13,0.157124,1.405337e+07,1.407525e+07,True,0.169015


### Derive ATR (10) for Volatility Adjustments

In [770]:
# Flatten MultiIndex columns if present to simplify DataFrame operations
tsla.columns = [
    '_'.join(col).strip() if isinstance(col, tuple) else col
    for col in tsla.columns
]

# Calculate the previous closing price to support True Range computation
tsla["prev_close"] = tsla["Close_TSLA"].shift(1)

# Compute three True Range variations used in ATR calculation
tsla["tr1"] = tsla["High_TSLA"] - tsla["Low_TSLA"]
tsla["tr2"] = abs(tsla["High_TSLA"] - tsla["prev_close"])
tsla["tr3"] = abs(tsla["Low_TSLA"] - tsla["prev_close"])

# Derive the True Range (TR) as the maximum of the three variants
tsla["true_range"] = tsla[["tr1", "tr2", "tr3"]].max(axis=1)

# Compute the 10-day Average True Range (ATR) to measure market volatility
tsla["ATR_10"] = tsla["true_range"].rolling(window=10).mean()

# Calculate ATR as a percentage of the current closing price to normalize volatility
tsla["atr_pct"] = tsla["ATR_10"] / tsla["Close_TSLA"]

# Define a function to assign position size based on volatility levels
def position_size(row):
    if row["atr_pct"] < 0.03:
        return 0.01  # Allocate 1% of capital for low-volatility conditions
    else:
        return 0.005  # Allocate 0.5% of capital for high-volatility conditions

# Apply the position size function across all rows
tsla["position_size"] = tsla.apply(position_size, axis=1)

# Display the latest 10 rows with selected indicators for inspection
print(tsla[["Close_TSLA", "ATR_10", "atr_pct", "position_size"]].tail(10))


            Close_TSLA     ATR_10   atr_pct  position_size
Date                                                      
2025-02-19  360.559998  16.703000  0.046325          0.005
2025-02-20  354.399994  16.464999  0.046459          0.005
2025-02-21  337.799988  17.021997  0.050391          0.005
2025-02-24  330.529999  16.770996  0.050740          0.005
2025-02-25  302.799988  18.879996  0.062351          0.005
2025-02-26  290.799988  18.412994  0.063318          0.005
2025-02-27  281.950012  18.257996  0.064756          0.005
2025-02-28  292.980011  18.067996  0.061670          0.005
2025-03-03  284.649994  19.281998  0.067739          0.005
2025-03-04  272.040009  20.654996  0.075926          0.005


In [771]:
# Create a line chart to visualize the ATR% (Average True Range as a percentage of price) over time
fig = px.line(
    tsla, 
    x=tsla.index, 
    y="atr_pct", 
    title="ATR% Over Time"  # Title of the chart
)

# Add a horizontal reference line at 3% to represent the low-volatility cutoff threshold
fig.add_hline(
    y=0.03, 
    line_dash="dot", 
    line_color="green", 
    annotation_text="Low Volatility Cutoff"
)

# Display the chart
fig.show()

/opt/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



The chart illustrates the historical volatility of TSLA, measured by the Average True Range (ATR) as a percentage of the closing price. Periods where the ATR% falls below the dotted green line at 3% indicate low volatility, which is typically associated with more stable market conditions. In contrast, noticeable spikes—such as those seen in 2020 and 2021—reflect periods of heightened volatility. More recently, ATR% values appear to remain closer to or slightly above the low-volatility threshold, suggesting relatively calmer market behavior compared to earlier years.

In [772]:
# Filter the TSLA DataFrame to include only records from the year 2025
tsla_2025 = tsla[tsla.index.year == 2025]

# Create a line chart to visualize ATR% for TSLA during 2025
fig = px.line(
    tsla_2025,
    x=tsla_2025.index,
    y="atr_pct",
    title="ATR% Over Time (2025 Only)"
)

# Add a horizontal line at the 3% threshold to denote the low-volatility cutoff
fig.add_hline(
    y=0.03,
    line_dash="dot",
    line_color="green",
    annotation_text="Low Volatility Cutoff"
)

# Display the chart
fig.show()

/opt/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



The chart displays ATR% for TSLA during 2025, reflecting how the stock's volatility has evolved since the start of the year. While ATR% began above the 7% mark in early January, it gradually declined and remained mostly between 4% and 6% throughout February. Although volatility did not breach the low-volatility threshold of 3%, the dip toward that level suggests a period of relative calm. Toward early March, ATR% showed a clear upward trend, indicating a potential resurgence in market volatility.

In [773]:
# Create Buy Signal
tsla['Buy_Signal'] = tsla['VI+_'] > tsla['VI-_']  # Vortex crossover

# Create Sell Signal (basic)
tsla['Sell_Signal'] = tsla['VI-_'] > tsla['VI+_']

# Initialize the position tracking column with 0 (no active position)
tsla['Position'] = 0

# Initialize a variable to store the peak price during a position for trailing stop logic
peak_price = 0

# Iterate through the dataset starting from index 1 to access previous values
for i in range(1, len(tsla)):

    # Entry condition: enter a position if a buy signal is present
    if tsla['Buy_Signal'].iloc[i]:
        tsla.at[tsla.index[i], 'Position'] = 1  # Mark entry into a position
        peak_price = tsla['Close_TSLA'].iloc[i]  # Record the entry price as initial peak

    # If already in position, check for exit condition using trailing stop
    elif tsla['Position'].iloc[i - 1] == 1:
        current_price = tsla['Close_TSLA'].iloc[i]  # Current closing price
        peak_price = max(peak_price, current_price)  # Update peak price if current exceeds previous
        drawdown = (peak_price - current_price) / peak_price  # Compute drawdown from peak

        # Exit condition: drawdown exceeds 3%
        if drawdown >= 0.03:
            tsla.at[tsla.index[i], 'Sell_Signal'] = True  # Trigger a sell signal
            tsla.at[tsla.index[i], 'Position'] = 0        # Exit position
        else:
            tsla.at[tsla.index[i], 'Position'] = 1        # Maintain position

# Display the total number of buy and sell signals generated across the dataset
print("Buy signals:", tsla['Buy_Signal'].sum())
print("Sell signals:", tsla['Sell_Signal'].sum())


Buy signals: 857
Sell signals: 680


In [774]:
# Create an empty figure object
fig = go.Figure()

# Plot the TSLA closing price as a continuous line
fig.add_trace(go.Scatter(
    x=tsla.index,
    y=tsla['Close_TSLA'],
    mode='lines',
    name='TSLA Price'
))

# Add markers to indicate Buy Signals using upward-pointing green triangles
fig.add_trace(go.Scatter(
    x=tsla[tsla['Buy_Signal']].index,
    y=tsla[tsla['Buy_Signal']]['Close_TSLA'],
    mode='markers',
    marker=dict(symbol='triangle-up', size=10, color='green'),
    name='Buy Signal'
))

# Add markers to indicate Sell Signals using downward-pointing red triangles
fig.add_trace(go.Scatter(
    x=tsla[tsla['Sell_Signal']].index,
    y=tsla[tsla['Sell_Signal']]['Close_TSLA'],
    mode='markers',
    marker=dict(symbol='triangle-down', size=10, color='red'),
    name='Sell Signal'
))

# Update layout settings including title and visual style
fig.update_layout(
    title='TSLA Buy & Sell Signals',
    template='plotly_white'
)

# Render the interactive plot
fig.show()

The chart illustrates the closing price of Tesla stock over time, with overlaid trading signals generated by the strategy. Green upward triangles represent buy signals, while red downward triangles mark sell signals. These signals are distributed throughout periods of both rising and falling prices, reflecting how the algorithm dynamically enters and exits positions based on market conditions. Clusters of signals during high-volatility periods—such as 2020, 2021, and early 2025—indicate frequent entries and exits, whereas more stable phases show fewer trades.

In [775]:
# Calculate ATR as a percentage of the closing price to normalize volatility
tsla['atr_pct'] = tsla['ATR_10'] / tsla['Close_TSLA']

# Define Vortex Indicator crossover signals:
# - VI_Cross_Up: Identifies when VI+ crosses above VI− (potential bullish signal)
# - VI_Cross_Down: Identifies when VI− crosses above VI+ (potential bearish signal)
tsla['VI_Cross_Up'] = (tsla['VI+_'] > tsla['VI-_']) & (tsla['VI+_'].shift(1) <= tsla['VI-_'].shift(1))
tsla['VI_Cross_Down'] = (tsla['VI-_'] > tsla['VI+_']) & (tsla['VI-_'].shift(1) <= tsla['VI+_'].shift(1))

# Initialize signal and state columns
tsla['Buy_Signal'] = False          # Flag for buy signal
tsla['Sell_Signal'] = False         # Flag for sell signal
tsla['Position'] = 0                # Position state: 1 = in position, 0 = no position
tsla['Entry_Type'] = None           # Strategy classification: 'aggressive' or 'conservative'

# Initialize control variables for trailing stop and price tracking
in_position = False                 # Boolean flag for current position state
peak_price = 0                      # Highest price observed during an open position

# Iterate through the DataFrame to simulate trading logic based on Vortex signals and volatility
for i in range(1, len(tsla)):
    row = tsla.iloc[i]
    idx = tsla.index[i]

    # Buy condition: Enter a new position if VI_Cross_Up occurs and no current position is held
    if not in_position and row['VI_Cross_Up']:
        tsla.at[idx, 'Buy_Signal'] = True
        tsla.at[idx, 'Position'] = 1
        in_position = True
        peak_price = row['Close_TSLA']

        # Classify entry type based on volatility threshold
        if row['atr_pct'] < 0.03:
            tsla.at[idx, 'Entry_Type'] = 'aggressive'
        else:
            tsla.at[idx, 'Entry_Type'] = 'conservative'

    # While in position, evaluate for trailing stop or VI_Cross_Down exit condition
    elif in_position:
        current_price = row['Close_TSLA']
        peak_price = max(peak_price, current_price)
        drawdown = (peak_price - current_price) / peak_price

        # Sell condition: Exit if drawdown exceeds 3% or VI_Cross_Down occurs
        if drawdown >= 0.03 or row['VI_Cross_Down']:
            tsla.at[idx, 'Sell_Signal'] = True
            tsla.at[idx, 'Position'] = 0
            in_position = False
        else:
            tsla.at[idx, 'Position'] = 1  # Maintain position

# Output the total count of each type of signal and entry classification
print("Buy signals:", tsla['Buy_Signal'].sum())
print("Sell signals:", tsla['Sell_Signal'].sum())
print("Aggressive entries:", (tsla['Entry_Type'] == 'aggressive').sum())
print("Conservative entries:", (tsla['Entry_Type'] == 'conservative').sum())

Buy signals: 80
Sell signals: 80
Aggressive entries: 5
Conservative entries: 75


In [776]:
# Create an empty figure to hold all plot layers
fig = go.Figure()

# Plot the TSLA closing price as a continuous blue line
fig.add_trace(go.Scatter(
    x=tsla.index,
    y=tsla['Close_TSLA'],
    mode='lines',
    name='TSLA Price',
    line=dict(color='blue')
))

# Add markers for aggressive buy signals (Entry_Type = 'aggressive')
fig.add_trace(go.Scatter(
    x=tsla[(tsla['Buy_Signal']) & (tsla['Entry_Type'] == 'aggressive')].index,
    y=tsla[(tsla['Buy_Signal']) & (tsla['Entry_Type'] == 'aggressive')]['Close_TSLA'],
    mode='markers',
    name='Buy (Aggressive)',
    marker=dict(symbol='triangle-up', color='limegreen', size=10)
))

# Add markers for conservative buy signals (Entry_Type = 'conservative')
fig.add_trace(go.Scatter(
    x=tsla[(tsla['Buy_Signal']) & (tsla['Entry_Type'] == 'conservative')].index,
    y=tsla[(tsla['Buy_Signal']) & (tsla['Entry_Type'] == 'conservative')]['Close_TSLA'],
    mode='markers',
    name='Buy (Conservative)',
    marker=dict(symbol='triangle-up', color='green', size=10)
))

# Add markers for sell signals using red downward-pointing triangles
fig.add_trace(go.Scatter(
    x=tsla[tsla['Sell_Signal']].index,
    y=tsla[tsla['Sell_Signal']]['Close_TSLA'],
    mode='markers',
    name='Sell Signal',
    marker=dict(symbol='triangle-down', color='red', size=10)
))

# Configure chart layout with appropriate title, axis labels, and style
fig.update_layout(
    title='TSLA Buy/Sell Signals Over Time',
    xaxis_title='Date',
    yaxis_title='Price (USD)',
    template='plotly_white',
    height=600
)

# Render the figure
fig.show()

The chart displays the historical closing price of Tesla (TSLA) stock alongside algorithmically generated buy and sell signals. The blue line represents TSLA's closing price, while the green upward-pointing triangles indicate buy entries—distinguished by lime green for aggressive entries (lower volatility) and dark green for conservative entries (higher volatility). Red downward-pointing triangles represent sell signals.

The buy signals are generally aligned with upward momentum, and sell signals frequently follow periods of short-term retracement or heightened volatility. The system shows particularly dense activity around highly volatile phases, such as mid-2020 to early 2022, capturing many entries and exits. In contrast, during more stable periods, the signals are more spaced out. Overall, the plot provides a clear visual assessment of how the strategy adapts dynamically to changing market conditions by modulating its entries based on volatility and exiting with protective trailing logic.

## Tesla Analysis Results

In [777]:
tsla_signals = tsla.reset_index()[['Date', 'VI_Cross_Up', 'VI_Cross_Down', 'atr_pct', 'Close_TSLA']]

In [778]:
merged_data = pd.merge(merged_data, tsla, on='Date', how='left')

In [779]:
# Calculate ATR percentage
merged_data['atr_pct'] = merged_data['ATR_10'] / merged_data['Close_TSLA']

# Vortex crossover logic
merged_data['VI_Cross_Up'] = (merged_data['VI+_'] > merged_data['VI-_']) & (merged_data['VI+_'].shift(1) <= merged_data['VI-_'].shift(1))
merged_data['VI_Cross_Down'] = (merged_data['VI-_'] > merged_data['VI+_']) & (merged_data['VI-_'].shift(1) <= merged_data['VI+_'].shift(1))

# Initialize signal & state columns
merged_data['Buy_Signal'] = False
merged_data['Sell_Signal'] = False
merged_data['Position'] = 0
merged_data['Entry_Type'] = None  # aggressive/conservative

# Trailing stop logic variables
in_position = False
peak_price = 0

for i in range(1, len(merged_data)):
    row = merged_data.iloc[i]
    idx = merged_data.index[i]
    # Buy condition
    if not in_position or row['VI_Cross_Up'] or row['5_day_avg_sentiment_norm']>0:
        merged_data.at[idx, 'Buy_Signal'] = True
        merged_data.at[idx, 'Position'] = 1
        in_position = True
        peak_price = row['Close_TSLA']

        # Entry Type: aggressive if ATR < 3%, else conservative
        if row['atr_pct'] < 0.03:
            merged_data.at[idx, 'Entry_Type'] = 'aggressive'
        else:
            merged_data.at[idx, 'Entry_Type'] = 'conservative'

    # While in position, check for trailing stop or VI cross down
    elif in_position:
        current_price = row['Close_TSLA']
        peak_price = max(peak_price, current_price)
        drawdown = (peak_price - current_price) / peak_price

        if drawdown >= 0.03 or row['VI_Cross_Down']:
            merged_data.at[idx, 'Sell_Signal'] = True
            merged_data.at[idx, 'Position'] = 0
            in_position = False
        else:
            merged_data.at[idx, 'Position'] = 1

# Show result counts
print("Buy signals:", merged_data['Buy_Signal'].sum())
print("Sell signals:", merged_data['Sell_Signal'].sum())
print("Aggressive entries:", (merged_data['Entry_Type'] == 'aggressive').sum())
print("Conservative entries:", (merged_data['Entry_Type'] == 'conservative').sum())


Buy signals: 18
Sell signals: 1
Aggressive entries: 0
Conservative entries: 18


In [780]:
# Ensure 'Date' is datetime and set as index if needed
merged_data['Date'] = pd.to_datetime(merged_data['Date'])

fig = go.Figure()

# Plot 5-day Avg Sentiment
fig.add_trace(go.Scatter(
    x=merged_data['Date'],
    y=merged_data['5_day_avg_sentiment_norm'],
    mode='lines+markers',
    name='5-Day Avg Sentiment',
    line=dict(color='blue')
))

# Plot ATR %
fig.add_trace(go.Scatter(
    x=merged_data['Date'],
    y=merged_data['atr_pct'],
    mode='lines+markers',
    name='ATR %',
    yaxis='y2',
    line=dict(color='orange')
))

# Optional: Highlight Buy Signal Dates (even though there are none now)
fig.add_trace(go.Scatter(
    x=merged_data.loc[merged_data['Buy_Signal'], 'Date'],
    y=merged_data.loc[merged_data['Buy_Signal'], '5_day_avg_sentiment_norm'],
    mode='markers',
    marker=dict(color='green', size=10, symbol='star'),
    name='Buy Signal'
))

# Add dual axis layout
fig.update_layout(
    title="5-Day Sentiment vs ATR % (with Buy Signals)",
    xaxis_title='Date',
    yaxis=dict(title='5-Day Avg Sentiment'),
    yaxis2=dict(title='ATR %', overlaying='y', side='right'),
    legend=dict(x=0.01, y=0.99),
    height=500
)

fig.show()


/opt/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [796]:
# Initialize portfolio variables
capital = 100000                   # Starting capital for the simulation
in_position = False               # Flag indicating whether a position is currently held
entry_price = 0                   # Entry price of the current position
position_value = 0                # Dollar value allocated to the position
cash = capital                    # Available cash (initially equal to capital)
returns = []                      # List to store profit/loss for each trade

# Iterate over the dataset to simulate trading
for i in range(len(merged_data)):
    row = merged_data.iloc[i]

    # ==== Buy Logic ====
    if row['Buy_Signal'] and not in_position:
        position_size = row['position_size']             # Fraction of capital to allocate
        position_value = cash * position_size            # Calculate how much capital to invest
        entry_price = row['Close_TSLA']                  # Record entry price
        shares_bought = position_value / entry_price     # Calculate number of shares to buy
        cash -= position_value                           # Deduct invested capital from cash
        in_position = True                               # Update position flag

    # ==== Sell Logic ====
    elif row['Sell_Signal'] and in_position:
        exit_price = row['Close_TSLA']                   # Get the exit price
        proceeds = shares_bought * exit_price            # Calculate proceeds from sale
        profit = proceeds - position_value               # Profit = proceeds - initial investment
        cash += proceeds                                 # Add proceeds back to cash
        returns.append(profit)                           # Record trade return
        in_position = False                              # Reset position state
        position_value = 0                               # Clear position value
        entry_price = 0                                  # Reset entry price

# ==== Final Capital Calculation ====
# If still holding a position, add unrealized value to cash
final_value = cash + (shares_bought * row['Close_TSLA'] if in_position else 0)
total_return = final_value - capital                    # Net profit/loss from strategy

# ==== Print Performance Metrics ====
print(f"Final Capital: ${final_value:,.2f}")
print(f"Total Return: ${total_return:.2f}")
print(f"Total Trades: {len(returns)}")
print(f"Average Profit per Trade: ${np.mean(returns):.2f}")

Final Capital: $99,898.47
Total Return: $-101.53
Total Trades: 1
Average Profit per Trade: $11.12


In [797]:
# Make sure index is datetime and 'Close_TSLA' exists
price = tsla['Close_TSLA']

# Generate entries and exits from your signals
entries = tsla['Buy_Signal']
exits = tsla['Sell_Signal']

# Create portfolio
portfolio = vbt.Portfolio.from_signals(
    close=price,
    entries=entries,
    exits=exits,
    size=np.nan,  # Let it auto-calculate position size if fixed capital
    init_cash=100_000,
    fees=0.001,  # 0.1% per trade
    slippage=0.0005  # Optional
)

In [798]:
# Summary stats
print(portfolio.stats())

# Equity curve
portfolio.plot().show()

/opt/anaconda3/lib/python3.11/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/opt/anaconda3/lib/python3.11/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/opt/anaconda3/lib/python3.11/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/opt/anaconda3/lib/python3.11/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2019-01-02 00:00:00
End                           2025-03-04 00:00:00
Period                                       1551
Start Value                              100000.0
End Value                                100000.0
Total Return [%]                              0.0
Benchmark Return [%]                  1215.813231
Max Gross Exposure [%]                        0.0
Total Fees Paid                               0.0
Max Drawdown [%]                              NaN
Max Drawdown Duration                         NaN
Total Trades                                    0
Total Closed Trades                             0
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                                  NaN
Best Trade [%]                                NaN
Worst Trade [%]                               NaN
Avg Winning Trade [%]                         NaN
Avg Losing Trade [%]                          NaN


In [799]:
print(tsla['Buy_Signal'].sum())  # Should be > 0
print(tsla['Sell_Signal'].sum())  # Should also be > 0

80
80


In [800]:
tsla = tsla.dropna(subset=['Close_TSLA'])
entries = tsla['Buy_Signal'].astype(bool)
exits = tsla['Sell_Signal'].astype(bool)

In [801]:
price = tsla['Close_TSLA']
portfolio = vbt.Portfolio.from_signals(
    close=price,
    entries=entries,
    exits=exits,
    init_cash=100_000,
    fees=0.001
)

print(portfolio.stats())
portfolio.plot().show()

Start                         2019-01-02 00:00:00
End                           2025-03-04 00:00:00
Period                                       1551
Start Value                              100000.0
End Value                           162759.235978
Total Return [%]                        62.759236
Benchmark Return [%]                  1215.813231
Max Gross Exposure [%]                      100.0
Total Fees Paid                      24054.581607
Max Drawdown [%]                        55.348959
Max Drawdown Duration                       730.0
Total Trades                                   80
Total Closed Trades                            80
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                                 32.5
Best Trade [%]                          46.283397
Worst Trade [%]                         -9.410141
Avg Winning Trade [%]                   11.344578
Avg Losing Trade [%]                    -3.847352


/opt/anaconda3/lib/python3.11/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/opt/anaconda3/lib/python3.11/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/opt/anaconda3/lib/python3.11/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/opt/anaconda3/lib/python3.11/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



The backtest results show that while the strategy achieved a total return of approximately 62.76%, it significantly underperformed compared to a simple buy-and-hold strategy on TSLA, which yielded a 1215.81% return. The strategy executed 80 trades with a low win rate of 32.5%, indicating that most trades were unprofitable. Although it had a few strong winners, the average profit per trade was marginal, with a profit factor of 1.19. Additionally, the portfolio experienced a substantial maximum drawdown of 55.35% and a prolonged recovery period lasting two years, signaling high risk. Visuals further confirm that many trades resulted in small losses or gains, with only a few notable profitable exits. Overall, while the strategy demonstrates some profitability, its risk-return profile is weak and may require optimization in entry/exit logic, volatility filtering, or sentiment integration to compete with the benchmark performance.